In [1]:
#general
import numpy as np
import pandas as pd
import re
import io

#vectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

#gridsearch
from sklearn.model_selection import cross_val_score, cross_val_predict, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator

#classifier
from sklearn.linear_model import LogisticRegression, SGDClassifier, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import BernoulliNB, CategoricalNB, ComplementNB, GaussianNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier, NearestCentroid
from sklearn.svm import LinearSVC, NuSVC, SVC

#words
from nltk.corpus import stopwords
from stop_words import get_stop_words
from itertools import combinations, combinations_with_replacement

import warnings
warnings.filterwarnings("ignore")

In [2]:
def print_to_string(*args, **kwargs):
    output = io.StringIO()
    print(*args, file=output, **kwargs)
    contents = output.getvalue()
    output.close()
    return contents

# Соревнование по сентимент-анализу
В этом соревновании вам предстоит прогнозировать по тексту отзыва его тональность: 1 - позитивная, 0 - негативная. В отличие от усложненной версии задачи, здесь вам не требуется самостоятельно собирать обучающую выборку - она есть в предоставляемых вам данных.

## Загрузка данных

В обучающей выборке на каждой строчке дано по одному тексту с классом (0 или 1), записанным через символ табуляции после текста.

In [3]:
df_train = pd.read_csv('data/products_sentiment_train.tsv', sep = '\t', header = None, names = ['text', 'class'])
df_test = pd.read_csv('data/products_sentiment_test.tsv', sep = '\t')

In [4]:
df_train.head()

,text,class
0,"2 . take around 10,000 640x480 pictures .",1
1,i downloaded a trial version of computer assoc...,1
2,the wrt54g plus the hga7t is a perfect solutio...,1
3,i dont especially like how music files are uns...,0
4,i was using the cheapie pail ... and it worked...,1


In [5]:
df_test.head()

,Id,text
0,0,"so , why the small digital elph , rather than ..."
1,1,3/4 way through the first disk we played on it...
2,2,better for the zen micro is outlook compatibil...
3,3,6 . play gameboy color games on it with goboy .
4,4,"likewise , i 've heard norton 2004 professiona..."


In [6]:
print('Количество объектов в обучающей выборке ', len(df_train))
print('Количество объектов в тестовой выборке ', len(df_test))
print('Количество неуникальных слов', sum(df_train['text'].str.split().apply(len)))
print('Среднее количество слов в документе', sum(df_train['text'].str.split().apply(len))/len(df_train))
print('Количество уникальных слов', TfidfVectorizer().fit_transform(df_train.text).shape[1])
print('Количество объектов класса 1 к количеству объектов в обучающей выборке равно ', sum(df_train['class'])/len(df_train))

Количество объектов в обучающей выборке  2000
Количество объектов в тестовой выборке  500
Количество неуникальных слов 39512
Среднее количество слов в документе 19.756
Количество уникальных слов 3973
Количество объектов класса 1 к количеству объектов в обучающей выборке равно  0.637


## Как будем обучать
Нам дали обучающую выборку с классами и тестовую без классов. Чтобы избежать переобучения, мы используем кросс валидацию на обучающей выборке. Согласно некоторой метрике (в рамках этой задачи accuracy) получаем самую лучшую модель, выбранную с помощью grid search и/или личных соображений/предпочтений. Чем больше качество на обучающей по вышеуказанному алгоритму, тем лучше оценка на тестовой выборке - по крайней мере после нескольких моих самбитов зависимость виднеется. 

В качестве метрики качества используется accuracy. При валидации качества не забывайте, что ваш результат точно должен быть лучше, чем тривиальные ответы (всегда 0, всегда 1, случайный выбор класса).

Особенности тестовой выборки мы не знаем - лишь предполагаем, что она похожа на обучающую выборку. Поэтому это зпдание чем-то похоже в игру в слепую)

## grid search - немного бездумный способ, который стоит применять с осторожностью
Не рекомендую на малом количестве данных, так как возможно *слегка* переобучиться 😢 Плюс из-за огромного количества параметров не особо удобно анализировать, от какого параметра изменяется метрика

### Создадим классы-обертки для удобной работы в pipeline

In [7]:
class DummyEstimator(BaseEstimator):
    def fit(self): pass
    def score(self): pass

class Doc2VecModel(BaseEstimator):
    def __init__(self, dm=1, vector_size=20, window=5, hs = 1, min_count = 5, epochs = 5, stop_words = None):
        self.d2v_model = None
        self.vector_size = vector_size
        self.window = window
        self.dm = dm
        self.hs = hs
        self.min_count = min_count
        self.epochs = epochs
        self.stop_words = stop_words

    def fit(self, raw_documents, y=None):
        # Initialize model
        self.d2v_model = Doc2Vec(vector_size=self.vector_size, 
                                window=self.window, 
                                dm=self.dm,
                                hs = self.hs,
                                min_count = self.min_count,
                                epochs = self.epochs)
        # Tag docs
        tagged_documents = []
        for index, row in raw_documents.iteritems():
            tag = '{}_{}'.format("type", index)
            tokens = row.split()
            if self.stop_words is not None:
                tokens = [word for word in tokens if word not in self.stop_words]
            tagged_documents.append(TaggedDocument(words=tokens, tags=[tag]))
        # Build vocabulary
        self.d2v_model.build_vocab(tagged_documents)
        # Train model
        self.d2v_model.train(tagged_documents, total_examples=len(tagged_documents), epochs=self.d2v_model.iter)
        return self

    def transform(self, raw_documents):
        X = []
        for index, row in raw_documents.iteritems():
            row = row.split()
            X.append(self.d2v_model.infer_vector(row))
        X = pd.DataFrame(X, index=raw_documents.index)
        return X

    def fit_transform(self, raw_documents, y=None):
        self.fit(raw_documents)
        return self.transform(raw_documents)        

### Создадим мега подборку параметров сетки, чтобы компьютер сгорел 🔥

In [8]:
comb = np.arange(1,11)
ngram_range = list(combinations_with_replacement(comb, 2))[:10]

In [9]:
#vect
vect_param_grid = {
    'vect': [CountVectorizer(), TfidfVectorizer()],
    'vect__min_df': np.arange(0.0, 1.1, 0.2),
    'vect__max_df': np.arange(0.0, 1.1, 0.2),
    'vect__ngram_range': ngram_range,
    'vect__stop_words': ['english', stopwords.words('english'), get_stop_words('en'), None],
    'vect__analyzer': ['word', 'char', 'char_wb']   
}
hashing_vect_param_grid = {
    'vect': [HashingVectorizer()],
    'vect__ngram_range': ngram_range,
    'vect__stop_words': ['english', stopwords.words('english'), get_stop_words('en'), None],
    'vect__analyzer': ['word', 'char', 'char_wb']   
}
doc2vec_param_grid = {
    'vect': [Doc2VecModel()],
    'vect__epochs': [1, 3, 5],
    'vect__hs': [0, 1],
    'vect__vector_size': [1, 10, 50, 100],
    'vect__dm':[0,1],
    'vect__window': np.arange(1,10,2),
    'vect__min_count': [0, 50, 100, 2000],
    'vect__stop_words': [stopwords.words('english'), get_stop_words('en'), None]
}

#clf
linear_param_grid = {
    'clf' : [LogisticRegression(), SGDClassifier(), RidgeClassifier()]
}
naive_param_grid = {
    'clf': [BernoulliNB(), CategoricalNB(), ComplementNB(), GaussianNB(), MultinomialNB()]
}
neighbors_param_grid = {
    'clf': [KNeighborsClassifier()]
}
forest_param_grid = {
    'clf': [RandomForestClassifier(), GradientBoostingClassifier()]
}
linear_svc_param_grid = {
    'clf':[LinearSVC()],
    'clf__loss': ['hinge', 'squared_hinge'],
    'clf__multi_class': ['ovr', 'crammer_singer'],
    'clf__class_weight': ['balanced', None],
    'clf__penalty': ['l1', 'l2']
}
svc_param_grid = {
    'clf':[SVC()],
    'clf__kernel': ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed'],
    'clf__class_weight': ['balanced', None]
}

In [10]:
param_grid = [
    vect_param_grid,
    hashing_vect_param_grid,
    doc2vec_param_grid,
    
    linear_param_grid,
    naive_param_grid,
    neighbors_param_grid,
    forest_param_grid,
    linear_svc_param_grid,
    svc_param_grid
]

### Запускаем долгий поиск по сетке

In [11]:
pipe = Pipeline(steps=[('vect', Doc2VecModel(vector_size = 1)), ('clf', LinearSVC(class_weight = None))])
search = GridSearchCV(pipe, param_grid, verbose = 1, scoring = 'accuracy', n_jobs=-1, cv = 5)
search = search.fit(df_train['text'], df_train['class'])

Fitting 5 folds for each of 11677 candidates, totalling 58385 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    8.9s
[Parallel(n_jobs=-1)]: Done 436 tasks      | elapsed:   16.5s
[Parallel(n_jobs=-1)]: Done 1136 tasks      | elapsed:   37.7s
[Parallel(n_jobs=-1)]: Done 1722 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 2512 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 3186 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 3936 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 5340 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 6408 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 7458 tasks      | elapsed:  5.2min
[Parallel(n_jobs=-1)]: Done 8608 tasks      | elapsed:  8.7min
[Parallel(n_jobs=-1)]: Done 9858 tasks      | elapsed: 12.6min
[Parallel(n_jobs=-1)]: Done 11208 tasks      | elapsed: 17.6min
[Parallel(n_jobs=-1)]: Done 12658 tasks     

In [80]:
def print_result(search, message = None):
    info = re.sub(' +', ' ',print_to_string('vect\n',search.best_estimator_['vect'], 
                                            '\nclf\n', search.best_estimator_['clf'],
                                            '\nscore\n', search.best_score_)
                 )
    print(info)
    
    message = '' if message is None else message + ' '
    name = message + str(search.best_score_) + ' ' + type(search.best_estimator_['vect']).__name__ + ' ' + type(search.best_estimator_['clf']).__name__
    
    df_test['y'] = pd.DataFrame(search.best_estimator_.predict(df_test['text']))
    
    df_test[['Id', 'y']].to_csv('results/' + name + '.csv', index=False)
    with open('results/' + name + '.txt', 'w', encoding='utf-8') as file:
        file.write(info)

In [81]:
print_result(search)

vect
 TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
 dtype=<class 'numpy.float64'>, encoding='utf-8',
 input='content', lowercase=True, max_df=1.0, max_features=None,
 min_df=0.0, ngram_range=(1, 10), norm='l2', preprocessor=None,
 smooth_idf=True, stop_words='english', strip_accents=None,
 sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
 tokenizer=None, use_idf=True, vocabulary=None) 
clf
 LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
 intercept_scaling=1, loss='squared_hinge', max_iter=1000,
 multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
 verbose=0) 
score
 0.7999999999999999



### Результаты
* Возможно переобучение. ngram_range получился сишком большим 1-10 слов, а чем больше этот диапозон, тем больше переобуечния. Поэтому оценка на тестовой выборке получилась 0.80444.
* Doc2Vec не справляется. Получилось добиться малой точности на обучающей выборке - 0.666. Этой модели банально не хватает данных - нейронная сеть DocVec не предназначена для таких маленьких выборок.
* Случайный лес и градиентный бустинг работают долго в силу своей специфики, но дают не такое хорошее качество, как простой и быстрый LinearSVC.
* TfidfVectorizer лучше CountVectorizer за счёт того, что вес некоторого слова пропорционален частоте употребления этого слова в документе и обратно пропорционален частоте употребления слова во всех документах коллекции

### Что плохого в grid search?
* Поиск по сетке ищет самое лучшее значение метрики, перебирая все возможные сочетания параметров. Это может привести к переообучению (кросс-валидация может ослабить переобучение, но оно возможно). 
* Неудобно анализировать, какие именно параметры вносят вклад - их много.

## Самостоятельный поиск 
В предыдущем примере было обнаружено, что самое лучшее значение даёт TfidfVectorizer, LinearSVC, LogisticRegression - попробуем развить идею.

In [82]:
param_grid = {
    'vect': [TfidfVectorizer()],
    'vect__ngram_range': ngram_range,
    'clf':[LinearSVC(),LogisticRegression()],    
    'clf__C': np.arange(0.6, 1.7 ,0.2)
}

In [83]:
pipe = Pipeline(steps=[('vect', TfidfVectorizer(ngram_range = [1,6])), ('clf', LinearSVC(C=1.0))])
search = GridSearchCV(pipe, param_grid, verbose = 1, scoring = 'accuracy', n_jobs=-1, cv = 5)
search = search.fit(df_train['text'], df_train['class'])

Fitting 5 folds for each of 120 candidates, totalling 600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    5.1s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   15.7s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   43.2s
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:  1.2min finished


In [84]:
print_result(search)

vect
 TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
 dtype=<class 'numpy.float64'>, encoding='utf-8',
 input='content', lowercase=True, max_df=1.0, max_features=None,
 min_df=1, ngram_range=(1, 3), norm='l2', preprocessor=None,
 smooth_idf=True, stop_words=None, strip_accents=None,
 sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
 tokenizer=None, use_idf=True, vocabulary=None) 
clf
 LinearSVC(C=1.4000000000000004, class_weight=None, dual=True,
 fit_intercept=True, intercept_scaling=1, loss='squared_hinge',
 max_iter=1000, multi_class='ovr', penalty='l2', random_state=None,
 tol=0.0001, verbose=0) 
score
 0.7915000000000001



Оценка на тестовой выборке этих параметров дала 0.81111

In [86]:
pd.DataFrame(
    {'params': search.cv_results_['params'],
    'score': search.cv_results_['mean_test_score']
    })

,params,score
0,"{'clf': LinearSVC(C=1.4000000000000004, class_...",0.7700
1,"{'clf': LinearSVC(C=1.4000000000000004, class_...",0.7850
2,"{'clf': LinearSVC(C=1.4000000000000004, class_...",0.7830
3,"{'clf': LinearSVC(C=1.4000000000000004, class_...",0.7780
4,"{'clf': LinearSVC(C=1.4000000000000004, class_...",0.7700
...,...,...
115,"{'clf': LogisticRegression(C=1.0, class_weight...",0.7230
116,"{'clf': LogisticRegression(C=1.0, class_weight...",0.7170
117,"{'clf': LogisticRegression(C=1.0, class_weight...",0.7085
118,"{'clf': LogisticRegression(C=1.0, class_weight...",0.7070


Через несколько самбитов удалось добиться точности 0.82000 на тестовой выборке с такими параметрами:
* TfidfVectorizer(ngram_range = `[1,6`])
* LinearSVC(C = 1.1)

### Увеличим обучающую выборку
Схитрим. Увеличим выборку, добавив тестовую выборку, которую разметили алгоритмом выше.

In [87]:
df_train_and_test = df_train.append(df_test[['text','y']].rename(columns={"y": "class"}))
df_train_and_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2500 entries, 0 to 499
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2500 non-null   object
 1   class   2500 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 58.6+ KB


In [88]:
pipe = Pipeline(steps=[('vect', TfidfVectorizer(ngram_range = [1,6])), ('clf', LinearSVC(C = 1.1))])
param_grid = {'vect': [TfidfVectorizer(ngram_range = [1,6])], 'clf': [LinearSVC(C = 1.1)]}
search = GridSearchCV(pipe, param_grid, verbose = 1, scoring = 'accuracy', n_jobs=-1, cv = 5)
search = search.fit(df_train_and_test['text'], df_train_and_test['class'])

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.5s remaining:    0.8s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.6s finished


In [89]:
print_result(search, 'df_train_and_test')

vect
 TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
 dtype=<class 'numpy.float64'>, encoding='utf-8',
 input='content', lowercase=True, max_df=1.0, max_features=None,
 min_df=1, ngram_range=[1, 6], norm='l2', preprocessor=None,
 smooth_idf=True, stop_words=None, strip_accents=None,
 sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
 tokenizer=None, use_idf=True, vocabulary=None) 
clf
 LinearSVC(C=1.1, class_weight=None, dual=True, fit_intercept=True,
 intercept_scaling=1, loss='squared_hinge', max_iter=1000,
 multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
 verbose=0) 
score
 0.8132000000000001



Точность на тестовой выборке стала хуже - 0.8066. То есть добавление не очень правильно размеченной выборки к обучающей выборке не добавляет качества.

## Итог
* выборка маленькая, из-за чего получить качество больше 80% сложновато.
* текст в датасете содержит грамматические ошибки, лишние символы, ошибки в построении предложений. Иногда встречаются нейтральные отзывы. Сама выборка не такая хорошая, как хотелось бы.
* данное соревнование больше походило на игру вслепую - мы стараемся добиться качества на тестовой выборке, зная лишь небольшой объём обучающей выборки. Из-за этого иногда качество на обучающей выборке не всегда равнялось качеству на тестовой - отклонения доходили до 2-4% в ту или иную сторону. Но всё равно чем больше было качество по кросс-валидации на обучающей выборке, тем больше было качество на тестовой (с некоторыми отклонениями от качества на обучающей).
* лучшим вариантом на таким объёмах является TFIDFVectorizer + LogisticRegression или LineraSVC.